## THIS NOTEBOOK WILL NOT WORK IN GOOGLE COLAB!!! IT MUST BE RUN LOCALLY!!! You can download it as an .ipynb file and open that with Jupyter notebook/labs

In [2]:
!pip install sqlalchemy
!pip install pymysql

import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd

If you get an error saying something like "sqlalchemy not installed", you should add a code cell and write `pip install sqlalchemy`. Then, rerun the above cell (you might also need to install pymysql the same way)

In [3]:
password = '/'
db_name = 'hotel_booking'

In [ ]:
# import os
# password = os.getenv("DB_PASSWORD")

In [4]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password=password, database=db_name)

In [5]:
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

In [30]:
reservations = pd.read_excel(r'C:\Users\llama\Downloads\Reservations.xlsx')

In [33]:
table_name = 'reservations'

In [34]:
reservations.to_sql(name=table_name, con=connection, if_exists='replace', index=False)

102

In [35]:
rooms = pd.read_excel(r'C:\Users\llama\Downloads\Rooms.xlsx')

In [36]:
table2_name = 'rooms'

In [37]:
rooms.to_sql(name=table2_name, con=connection, if_exists='replace', index=False)

50

In [32]:
guests = pd.read_excel(r'C:\Users\llama\Downloads\Guests1.xlsx')

In [33]:
table3_name = 'guests'

In [34]:
guests.to_sql(name=table3_name, con=connection, if_exists='replace', index=False)

101

In [6]:
payments = pd.read_excel(r'C:\Users\llama\Downloads\Payments1.xlsx')

In [7]:
table4_name = 'payments'

In [8]:
payments.to_sql(name=table4_name, con=connection, if_exists='replace', index=False)

102

We use SQL syntax for querying. The triple quotation mark syntax allows us to write commands simiar to how we did in SQL

In [35]:
query1 = "SELECT * FROM guests"
query2 = """

SELECT * FROM guests
WHERE Membership_Status ='Platinum'

"""

In [36]:
result_df = pd.read_sql(query1, con=connection)

In [37]:
result_df

,First_Name,Last_Name,Guest_ID,Address,Phone_number,Email,Membership_Status
0,Kevin,Thompson,1000,"165 Parrish Stravenue Apt. 494, Dalehaven, FL ...",(478) 686-3375,fmiles@hotmail.com,Silver
1,Travis,Williams,1001,"8514 Lindsay Vista, New Sarahview, HI 36319",(069) 033-1190,christopher18@mccann.net,Silver
2,Colton,Harmon,1002,"5022 Jones Streets Apt. 183, Barnesmouth, MN 4...",(067) 383-0284,wwillis@hotmail.com,Standard
3,Paige,Fox,1003,"22901 Ramos Manors, Lawsonchester, IL 99452",(342) 836-4408,holly30@miller.com,Standard
4,Derek,Reese,1004,"1497 Williams Locks Apt. 034, Port Derrick, LA...",(056) 856-6246,parkerjudith@hotmail.com,Standard
...,...,...,...,...,...,...,...
96,Lindsey,Hines,1097,"135 Julie Inlet Suite 554, Stevenville, AR 83652",991-738-1684,pittsedward@gmail.com,Platinum
97,Matthew,Benitez,1098,"52175 Lawrence Centers Apt. 743, New Kathy, WY...",966-667-6008,stephanie58@smith.com,Standard
98,Jeremy,Wiley,1099,"8942 Madison Hills, Stephenhaven, NC 44781",257-532-6033,richardsondebra@brooks.com,Platinum
99,Kelsey,Dean,1100,"100 Robert Light, New Johnnyberg, UT 90382",046-184-9926,glovertyler@hotmail.com,Platinum


In [38]:
result_df_2 = pd.read_sql(query2, con=connection)

In [39]:
result_df_2

,First_Name,Last_Name,Guest_ID,Address,Phone_number,Email,Membership_Status
0,Robert,Logan,1012,"51864 Cummings Shoals Apt. 519, West Kyle, UT ...",(808) 010-0623,margaret95@gmail.com,Platinum
1,Ricky,Ellis,1015,"880 Walter River, New Nicholas, HI 91313",(398) 454-6156,kellymueller@hotmail.com,Platinum
2,Antonio,Brown,1017,"59774 Shaw Manor Apt. 392, Brettfort, MN 73185",(659) 180-3397,randolphandrew@reed.com,Platinum
3,Mitchell,Mendoza,1018,"USS Mitchell, FPO AE 36256",(890) 250-3926,usweeney@garcia-moreno.com,Platinum
4,Cory,Browning,1022,"17980 Jennifer Highway Suite 095, New Robert, ...",(655) 110-4754,kjones@graham.com,Platinum
5,Amy,Miller,1024,"05852 Pollard Neck Apt. 217, West Jamesborough...",(604) 661-8398,jesus32@hotmail.com,Platinum
6,Tiffany,Franklin,1025,"USCGC Hays, FPO AP 62550",(444) 721-2646,danielcole@russell-herrera.com,Platinum
7,Michael,Jones,1026,"5054 Lauren Mews, Paulfurt, HI 65539",(437) 993-7124,steven11@gmail.com,Platinum
8,Travis,Williams,1029,"8515 Lindsay Vista, New Sarahview, HI 36319",(069) 033-1191,christopher18@mccann.net,Platinum
9,Kristin,Miles,1036,"84249 Wayne Extension Apt. 299, Meganshire, AK...",(362) 266-0937,patrick63@wilkerson.net,Platinum
